# Cuaderno 14: Problema del agente viajero asimétrico 
# Asymmetric Traveling Salesman Problem (ATSP)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo dirigido **completo** $D=(V,A)$; y,
* un vector $c \in \ZZ^{A}$ de costos asociados a los arcos de $D$.

El *problema del agente viajero asimétrico (Asymmetric Traveling Salesman Problem, ATSP)* consiste en encontrar un circuito (llamado también *tour*) que visite **todos** los nodos de $D$ y que tenga el menor costo posible. En el problema del agente viajero simétrico *(TSP)* requerimos adicionalmente que $c_{ij} = c_{ji}$ se cumpla para todo par de nodos $i,j \in V$.

El problema del agente viajero (en sus versiones simétrica y asimétrica) es un problema clásico de la optimización combinatoria que ha despertado históricamente mucho interés. Al ser un problema NP-difícil, no se conoce un algoritmo eficiente para su solución en general. El estudio de este problema ha conducido a avances significativos en el campo de la combinatoria poliedral y en el desarrollo de métodos de solución basados en la programación lineal entera. Mayor información sobre este tópico está disponible [en el siguiente enlace](https://www.math.uwaterloo.ca/tsp/index.html). En el mismo sitio web está además disponible el solver [Concorde](https://www.math.uwaterloo.ca/tsp/concorde/index.html), que actualmente constituye una de las herramientas computacionales más rápidas para la solución exacta de instancias del TSP. 

Utilizando variables binarias $x_{ij}$ para indicar la selección de arcos en el circuito, el problema del agente viajero puede formularse como el siguiente programa lineal entero:

\begin{align*}
\min &\sum_{(i,j) \in A} c_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
&\sum_{(j, i) \in A} x_{ji} = 1, \quad \forall i \in V,\\
&\sum_{(i, j) \in A} x_{ij} = 1, \quad \forall i \in V,\\ 
&\sum_{(i,j) \in E(W)} x_{ij} \leq \card{W} - 1, \quad \forall W \subset V, \emptyset \neq W \neq V,\\
& x_{ij} \in \{0, 1\}, \quad \forall (i, j) \in A.
\end{align*}
donde $E(W) := \{ (i,j) \in A \, : \, i \in W, j \in W\}$.

La función objetivo mide el costo total de los arcos seleccionados en la solución.

Las dos primeras familias contienen restricciones de grado. Estas especifican que, para cada nodo del grafo, debe seleccionarse un arco entrante y un arco saliente.

Además de los toures factibles del problema del agente viajero, puede demostrarse que las únicas soluciones que satisfacen las restricciones de grado consisten de circuitos disjuntos que cubren todos los nodos del grafo. La tercera familia de restricciones se emplea para eliminar estas últimas soluciones. Para cualquier subconjunto propio de nodos $W$, se requiere que la cantidad de aristas seleccionadas que tengan sus dos extremos en $W$ debe ser menor o igual a $\card{W} -1$. Debido a la función que cumplen, estas restricciones se conocen como *restricciones de eliminación de subtoures*. Notar que hay un restricción de este tipo por cada subconjunto propio no vacío de nodos, es decir, $2^n - 2$ restricciones para un grafo con $n$ nodos.

Vamos a implementar este modelo usando la interfaz Python de Gurobi.



Definimos primero los datos. Usaremos la función `randint` para generar puntos con coordenadas aleatorias en el rango {0,..,100}. El costo de un arco $(i,j)$ será igual a la distancia euclideana entre $i$ y $j$:

In [ ]:
from gurobipy import *
import random as rm
import math 

# Numero de nodos del grafo
n = 10

# Nodos del grafo
V = tuplelist(range(1,n+1))

# Posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
 
A = c.keys()


Vamos a necesitar construir una restricción de eliminación de subciclos para cada subconjunto de nodos $W \subset V$ tal que $\emptyset \neq W \neq V$. Con esta finalidad, definimos una función `powerset` empleando las funciones `chain` y `combinations` del módulo `itertools`.

In [ ]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


Definimos ahora el objeto modelo, las variables y la función objetivo:

In [ ]:
# Crear el objeto modelo
m = Model('atsp')

# Crear las variables
x = m.addVars(A, name="x", vtype=GRB.BINARY)

# Crear la funcion objetivo
m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)


Añadimos las restricciones de grado:

In [ ]:
# Restricciones de grado saliente
m.addConstrs((x.sum(i,'*')  == 1 for i in V), 
                 "g_saliente")
    
# Restricciones de grado entrante
m.addConstrs((x.sum('*', i)  == 1 for i in V), 
                 "g_entrante")

Para construir las restricciones de subciclos, usamos la función `powerset` para iterar sobre todos los subconjuntos de vértices $W$ tales que $\emptyset \neq W \neq V$. Para cada subconjunto, la expresión `x.sum(W, W)` construye la suma de las variables asociadas a arcos con ambos extremos en $W$.

In [ ]:
# Restricciones de eliminación de subciclos
for W in powerset(V):
    if W!=() and W!=tuple(V):
        m.addConstr(x.sum(W, W) <= len(W)-1, "subciclo[{}]".format(W))


Escribimos el modelo a un archivo de texto:

In [ ]:
# Escribir el modelo a un archivo
m.write('tsp.lp')


Finalmente, resolvemos el modelo y mostramos la solución:

In [ ]:
# Calcular la solucion optima
m.optimize()

# Escribir la solucion
if m.status == GRB.Status.OPTIMAL:
    # Recuperar los valores de las variables
    vx = m.getAttr('x', x)
    print('\nTour optimo:')
    for i,j in A:
        if vx[i,j] >= 0.99:
            print('{} -> {}'.format(i, j))


### Graficando los toures

Podemos emplear el módulo `matplotlib` para graficar el tour de la solución. Definiremos para ello la función `dibujarTour` que recibe tres argumentos: una lista `coordx` con las coordenadas horizontales de los nodos, una lista `coordy` con las coordenadas verticales y un vector `tour` con una permutación de los nodos indicando el orden de visita en la solución.

In [ ]:
import matplotlib.pyplot as plt
import random

def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour]
    Ty = [coordy[i] for i in tour]
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
        plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=2)
    plt.show()


Utilicemos ahora esta función para graficar la solución de nuestra instancia:

In [ ]:
# Crear lista con arcos seleccionados en la solucion
L = [(i,j) for i,j in A if vx[i,j]>=0.9]
print(L)

# Recuperar el tour como un ordenamiento de los nodos
T = [1]
# nodo actual:
i = 1
while True:
    # Determinar sucesor de i
    j = [j for j in V if (i,j) in L].pop()
    # Colocar sucesor en la lista del tour y actualizar i
    T.append(j)
    i = j
    # Terminar cuando el nodo colocado sea 1
    if i==1: break;

print(T)
        
# Graficar el tour
dibujarTour(coordx, coordy, T)    

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementación de modelos lineales enteros
# Problema del agente viajero (TSP)
# Luis M. Torres (EPN 2021)

from gurobipy import *
import random as rm
import matplotlib.pyplot as plt
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour]
    Ty = [coordy[i] for i in tour]
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
        plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=2)

# Numero de nodos del grafo
n = 20

# Nodos del grafo
V = tuplelist(range(1,n+1))

# Posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
 
A = c.keys()

try:
    # Crear el objeto modelo
    m = Model('tsp')

    # Crear las variables
    x = m.addVars(A, name="x", vtype=GRB.BINARY)

    # Crear la funcion objetivo
    m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

    # Restricciones de grado saliente
    m.addConstrs((x.sum(i,'*')  == 1 for i in V), 
                 "g_saliente")
    
    # Restricciones de grado entrante
    m.addConstrs((x.sum('*', i)  == 1 for i in V), 
                 "g_entrante")

    # Restricciones de eliminacion de subciclos
    for W in powerset(V):
        if W!=() and W!=tuple(V):
            m.addConstr(x.sum(W, W) <= len(W)-1, 
                        "subciclo[{}]".format(W))

    # Escribir el modelo a un archivo
    # m.write('tsp.lp')

    # Calcular la solucion optima
    m.optimize()

    # Escribir la solucion
    if m.status == GRB.Status.OPTIMAL:
        # Recuperar los valores de las variables
        vx = m.getAttr('x', x)
        print('\nTour optimo:')
        for i,j in A:
            if vx[i,j] > 0:
                print('{} -> {}'.format(i, j))
                
    # Crear lista con arcos seleccionados en la solucion
    L = [(i,j) for i,j in A if vx[i,j]>0]

    # Recuperar el tour como un ordenamiento de los nodos
    T = [1]
    # nodo actual:
    i = 1
    while True:
        # Determinar sucesor de i
        j = [j for j in V if (i,j) in L].pop()
        # Colocar sucesor en la lista del tour y actualizar i
        T.append(j)
        i = j
        # Terminar cuando el nodo colocado sea 1
        if i==1: break;
        
    # Graficar el tour
    dibujarTour(coordx, coordy, T)    
    
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')

### Creando el conjunto potencia de un conjunto en Python

Para construir el conjunto potencia (la familia de subconjuntos) de cualquier iterable, pueden usarse las funciones `chain` y `combinations` del módulo `itertools`.

In [ ]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

V = list([1, 2, 3, 4, 5])
P = list(powerset(V))
print("V= {}".format(V))
print("P= {}".format(P))

for W in powerset(V):
    if W!=() and W!=tuple(V):
        print (W, len(W))

In [ ]:
L = powerset([1,2,3,4])
for i in L:
    print(i)